# TruthLens - Data Collection

TruthLens is a project developed for the BSc. Computer Science (Data Science) Final Project (CM3070) at the University of London. TruthLens is based on the Fake News Detection template. 

## Project Objectives
The primary objective of this project is to build a two-stage pipeline for misinformation classification:

1. Binary classification (Stage 1): Distinguish between real news and misinformation using the ISOT dataset. This ensures robust detection at the first stage, leveraging an established dataset.
2. Multi-class classification (Stage 2): Further classify content identified as misinformation into one of seven categories, based on Molina et al.’s taxonomy. A custom dataset will support this nuanced classification.

The scope of the project is limited to text-based, English language content, explicitly excluding images and videos. A user interface will also be developed, enabling users to input articles or URLs and receive classification results.

A secondary objective is to enhance the explainability of classification results, aiming to provide users with interpretable insights into why content was classified in a particular way.

The project aims for high accuracy and reliability, with measurable performance goals. Ethical considerations, including bias mitigation and responsible dataset usage, will guide the design and implementation of the pipeline.

## Custom dataset generation
As outlined in the previous section, the second stage of the pipeline relies upon a custom dataset, labelled with the categories from the Molina et al. Misinformation Taxonomy. These classes are summarised in the table below. The aim of this stage is to create a balanced dataset with 200 pieces of content for each of the 7 categories. 

| Misinformation Type | Characteristics | Example |
|:--------------|:---------------|:-------|
| Fabricated content | Completely false content created with the intent to deceive.| Fake reports of events that never occurred; entirely false claims about public figures |
|Polarised content |True events or facts presented selectively to promote a biased narrative, often omitting critical context. |Partisan news articles highlighting one side of a political argument while ignoring counterpoints.|
|Satire |Content intended to entertain or provoke thought through humour, exaggeration, or irony. Often misunderstood. |Satirical articles from outlets like “The Onion” being shared as if they are factual news.|
|Misreporting | Incorrect information shared unintentionally, often due to errors or lack of verification. | A news outlet incorrectly reporting election results due to early or inaccurate data.|
|Commentary |Opinion-based content reflecting the writer’s interpretation or viewpoint, often lacking factual grounding. |Editorials or blogs expressing subjective opinions without substantial evidence.|
|Persuasive information |Content designed to persuade or influence the audience, often including marketing and propaganda. |Politically motivated propaganda campaigns, advertisements disguised as objective news articles.|
|Citizen journalism | User-generated content that may lack professional journalistic standards, leading to error or bias. |Social media posts about breaking news that spread unverified or incorrect details.|

Data will be scrapped from relevant websites for each category, then manually reviewed to ensure that it fits the category. Relevant features and labelling guidelines can be found for each category below.

In [1]:
#Imports and helper functions
import requests
import json
from bs4 import BeautifulSoup
import csv
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re
import string
import nltk
from nltk.corpus import stopwords

def preprocess_text(text):
    """
        Preprocesses a given text string by applying the following steps:
        1. Converts the text to lowercase.
        2. Removes punctuation marks.
        3. Tokenizes the text into individual words.
        4. Removes stopwords (common words that add little value to classification tasks).

        Parameters:
        ----------
        text : str
            The input text string to preprocess.

        Returns:
        -------
        str
            The cleaned and preprocessed text, with tokens joined back into a single string.
    """
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [word for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

### 2. Polarised content
Polarised content is true events or facts selectively presented to promote a biased narrative, often omitting critical context.

##### Features:
- Partial Truth: The piece is based on a real event, statistic, or quote.
- Omission / Distortion: The content emphasizes certain facts while ignoring or minimizing others, creating a skewed impression.
- Strong Bias: The language or framing clearly supports one political, ideological, or partisan stance, rather than offering balanced coverage.

##### Label if:
- The article references real events but uses them to push a strong, one-sided narrative.
- The content focuses on data or testimonies that bolster a specific stance while disregarding contradictory evidence.
- The tone or style is heavily partisan and attempts to sway opinion by selective fact usage rather than outright fabrication.

##### Do Not Label if:
- The core facts are outright false (label as Fabricated).
- It is primarily personal opinion or commentary without strong factual references (label as Commentary).
- It is purely an attempt at persuasion or advertising without misrepresenting an event (label as Persuasive).

##### Sources:
- The Conservative Woman (UK, Right leaning) https://www.conservativewoman.co.uk/
- The Canary (UK, Left leaning) https://www.thecanary.co/uk/
- Breitbart (USA, Right leaning) https://www.breitbart.com/
- Daily Kos (USA, Left leaning) https://www.dailykos.com/

**The Conservative Woman**

Articles were scraped from the weekly "Our Top Ten Articles of the Week" series, starting from the January 11, 2025 edition (https://www.conservativewoman.co.uk/tcw-our-top-ten-articles-of-the-week-9/). 

A large number of articles were skipped. "Features" and "Family and Faith" articles were skipped as they are not news. Many of the other articles did not meet the criteria for labelling, instead falling under Commentary, for example: https://www.conservativewoman.co.uk/wind-turbines-and-a-voice-in-the-wilderness/ These were primarily recognised by a focus on "I" and "me" in the text.

In [28]:
def scrape_tcw_article(url):
    """
    Scrapes an article from a given URL on conservativewoman.co.uk and extracts relevant information.

    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Polarised", 
        "url": url
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # Title
        title_meta = soup.find('meta', property='og:title')
        article_data["title"] = title_meta['content'] if title_meta else "Title not found"
        # Remove the trailing site name
        if article_data["title"].endswith(" - The Conservative Woman"):
            article_data["title"] = article_data["title"].replace(" - The Conservative Woman", "")
        
        # URL
        url_meta = soup.find('meta', property='og:url')
        article_data["url"] = url_meta['content'] if url_meta else url  # Fallback to input URL
        
        # Site name
        site_name_meta = soup.find('meta', property='og:site_name')
        article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
        
        # Published date
        published_date_meta = soup.find('meta', property='article:published_time')
        article_data["date"] = published_date_meta['content'] if published_date_meta else "Published date not found"
        
        # Category
        yoast_script = soup.find("script", class_="yoast-schema-graph", type="application/ld+json")
        if yoast_script:
            try:
                data_json = json.loads(yoast_script.string)
                for node in data_json.get("@graph", []):
                    if node.get("@type") == "Article":
                        art_sec = node.get("articleSection", None)
                        if art_sec:
                            if isinstance(art_sec, list):
                                article_data["category"] = art_sec[0]
                            else:
                                article_data["category"] = art_sec
                        break
            except json.JSONDecodeError:
                print("Could not parse the JSON-LD correctly.")
        
        # Article copy
        content_div = soup.find("div", class_=lambda c: c and "td-post-content" in c)
        if content_div:
            # Collect paragraphs
            paragraphs = content_div.find_all("p")
            text_list = []
            for p in paragraphs:
                text = p.get_text(strip=True)
                # End before the donation paragraph
                if text.startswith("If you appreciated this article, perhaps you might consider making a donation"):
                    break  
                text_list.append(text)
            #join all paragraphs together
            full_text = " ".join(text_list).strip()
            # Remove web addresses using a regex
            full_text = re.sub(r'https?://\S+', '', full_text)    
            article_data["text"] = full_text
        else:
            article_data["text"] = "Article text not found"
    
    else:
        print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")

    return article_data



def scrape_multiple_tcw_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_tcw_article(url)
        articles.append(article)
    return pd.DataFrame(articles)


# List of URLs to scrape
urls = ["https://www.conservativewoman.co.uk/the-uk-grooming-gang-scandal-is-a-galileo-moment/",
        "https://www.conservativewoman.co.uk/progressive-contempt-for-the-white-working-class/",
        "https://www.conservativewoman.co.uk/how-dare-starmer-reject-a-public-inquiry-into-muslim-grooming-gangs/",
        "https://www.conservativewoman.co.uk/a-quad-demic-christmas-blown-out-of-proportion-and-a-happy-new-year/",
        "https://www.conservativewoman.co.uk/how-labour-is-seizing-more-control-over-our-children/",
        "https://www.conservativewoman.co.uk/david-keighley-multiculturalism-slays-rotherhams-young-girls/",
        "https://www.conservativewoman.co.uk/nelson-turns-a-blind-eye-to-broken-britain/",
        "https://www.conservativewoman.co.uk/major-study-confirms-covid-jab-harms-mental-health/",
        "https://www.conservativewoman.co.uk/why-was-saras-sadistic-murderous-father-not-stopped-we-all-know-why/",
        "https://www.conservativewoman.co.uk/stories-from-the-illegal-migrant-frontline/",
        "https://www.conservativewoman.co.uk/the-climate-scaremongers-met-office-fiddles-the-figures-over-storm-darragh/",
        "https://www.conservativewoman.co.uk/electric-armoured-vehicles-net-zero-chance-of-that/",
        "https://www.conservativewoman.co.uk/war-on-microbes-the-murky-agenda-behind-the-covid-pandemic/",
        "https://www.conservativewoman.co.uk/this-methane-nonsense-is-a-nasty-protection-racket/",
        "https://www.conservativewoman.co.uk/why-nhs-staff-are-shunning-the-vaccines/",
        "https://www.conservativewoman.co.uk/why-report-the-mass-rape-of-white-schoolgirls-when-you-can-pick-on-gregg-wallace/",
        "https://www.conservativewoman.co.uk/debunked-the-great-diversity-equity-and-inclusion-myth/",
        "https://www.conservativewoman.co.uk/marvel-at-trumps-resurgent-america-weep-for-starmers-desolate-britain/",
        "https://www.conservativewoman.co.uk/exit-june-raine-pursued-by-bare-faced-lies/",
        "https://www.conservativewoman.co.uk/from-1991-a-prescient-warning-about-the-globalist-agenda/",
        "https://www.conservativewoman.co.uk/beware-sir-keir-beware-this-petition-is-the-tip-of-the-iceberg/",
        "https://www.conservativewoman.co.uk/the-blackrock-connection-and-a-nightmare-for-farmers/",
        "https://www.conservativewoman.co.uk/one-of-the-worlds-oldest-christian-communities-faces-destruction/",
        "https://www.conservativewoman.co.uk/covid-didnt-cause-surge-in-excess-deaths-the-pandemic-response-did/",
        "https://www.conservativewoman.co.uk/a-book-to-destroy-faith-in-doctors-for-ever/",
        "https://www.conservativewoman.co.uk/staggering-ignorance-that-scuppered-sterling-and-the-stock-exchange/",
        "https://www.conservativewoman.co.uk/oxbridge-and-the-cancellation-of-kindness/",
        "https://www.conservativewoman.co.uk/msm-silence-as-health-coalition-urges-governments-stop-the-jabs-now/",
        "https://www.conservativewoman.co.uk/methane-reducing-feed-additive-trialled-in-arla-dairy-farms/",
        "https://www.conservativewoman.co.uk/still-time-to-pull-back-from-slippery-suicide-slope/",
        "https://www.conservativewoman.co.uk/this-ghastly-assisted-suicide-bill-strikes-against-decency-and-genuine-choice/",
        "https://www.conservativewoman.co.uk/cop29-reveals-itself-as-the-great-fraud-it-always-was/",
        "https://www.conservativewoman.co.uk/david-keighley-was-right-everything-he-warned-about-hate-crime-has-come-to-pass/",
        "https://www.conservativewoman.co.uk/the-climate-scaremongers-bbc-admits-it-lied-about-vanishing-polar-bears/",
        "https://www.conservativewoman.co.uk/a-vaccine-guinea-pigs-heartrending-chronicle-of-a-life-destroyed/",
        "https://www.conservativewoman.co.uk/muslim-rioters-rampage-with-police-blessing-today-amsterdam-tomorrow-britain/",
        "https://www.conservativewoman.co.uk/a-warning-shot-writ-large-but-putin-wont-attack-the-west/",
        "https://www.conservativewoman.co.uk/my-choice-as-world-leader-of-the-century-netanyahu/",
        "https://www.conservativewoman.co.uk/badenoch-should-listen-to-clarkson-and-learn/",
        "https://www.conservativewoman.co.uk/you-must-be-bad-if-even-the-amish-are-up-in-arms/",
        "https://www.conservativewoman.co.uk/trumps-multiethnic-winning-coalition/",
        "https://www.conservativewoman.co.uk/the-billions-upon-billions-wasted-on-useless-face-masks/",
        "https://www.conservativewoman.co.uk/why-the-law-is-stacked-against-right-thinkers/",
        "https://www.conservativewoman.co.uk/whats-the-real-reason-theyre-going-after-allison-pearson/",
        "https://www.conservativewoman.co.uk/health-warrior-rfk-jr-faces-coalition-of-formidable-enemies/",
        "https://www.conservativewoman.co.uk/so-cruel-so-vulnerable-the-daycare-generation/",
        "https://www.conservativewoman.co.uk/killing-freedom-under-the-banner-of-public-health/",
        "https://www.conservativewoman.co.uk/the-climate-scaremongers-energy-operator-tells-miliband-your-plans-cannot-work/",
        "https://www.conservativewoman.co.uk/here-is-the-long-term-weather-report-same-old-same-old/",
        "https://www.conservativewoman.co.uk/revealed-pfizers-hidden-vaccine-injuries/",
]

# Provide a common browser user agent - otherwise the scraping fails
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    )
}

# Scrape articles and create a DataFrame
tcw_data_df = scrape_multiple_tcw_articles(urls)
# Store to CSV
tcw_data_df.to_csv("polarised_scraped_articles_tcw.csv", index=False)
# Print head 
tcw_data_df.head()

title  \
0                      The UK grooming gang scandal is a Galileo moment   
1                The progressives' contempt for the white working class   
2  How dare Starmer reject a public inquiry into Muslim grooming gangs?   
3                               Welcome to the year of the 'Quad-demic'   
4                  How Labour is seizing more control over our children   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

**The Canary**

Articles have been scraped from the UK section of The Canary (https://www.thecanary.co/uk/) from newest to oldest. Article date range is January 19th to January 7th. Two articles were excluded for not meeting the labelling criteria (articles focused on getting users to sign a petition.)

In [33]:
testURL = "https://www.thecanary.co/uk/analysis/2025/01/19/fii-long-covid/"

def scrape_can_article(url):
    """
    Scrapes an article from a given URL on https://www.thecanary.co/uk/ and extracts relevant information.
    
    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Polarised",
        "url": url
    }

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Remove tweet embeds
            for twitter_blockquote in soup.find_all('blockquote', class_='twitter-tweet'):
                twitter_blockquote.decompose()
            # Remove ad elements
            for ads_div in soup.find_all('div', class_='ads_google_ads'):
                ads_div.decompose()

            # Title
            title_meta = soup.find('meta', property='og:title')
            article_data["title"] = title_meta['content'] if title_meta else "Title not found"
            
            # URL
            url_meta = soup.find('meta', property='og:url')
            article_data["url"] = url_meta['content'] if url_meta else url
            
            # Site name
            site_name_meta = soup.find('meta', property='og:site_name')
            article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
            
            # Published date
            published_date_meta = soup.find('meta', property='article:published_time')
            article_data["date"] = published_date_meta['content'] if published_date_meta else "Published date not found"
            
            # Category
            category_found = None
            yoast_script = soup.find('script', class_='yoast-schema-graph', type='application/ld+json')
            if yoast_script:
                try:
                    yoast_data = json.loads(yoast_script.string)
                    for item in yoast_data.get('@graph', []):
                        if item.get('@type') == 'NewsArticle':
                            section = item.get('articleSection')
                            if section:
                                if isinstance(section, list) and len(section) > 0:
                                    category_found = section[0].strip()
                                elif isinstance(section, str):
                                    category_found = section.strip()
                                break
                except json.JSONDecodeError:
                    pass
            # If we never found a category, use a default
            if category_found:
                article_data["category"] = category_found
            else:
                article_data["category"] = "Category not found"
            
            # Article copy
            article_body = soup.find('div', class_='jeg_inner_content')
            featured_image_patterns = [
                re.compile(r'^Featured image via .*$', re.IGNORECASE),
                re.compile(r'^Featured image supplied', re.IGNORECASE),
                re.compile(r'^Featured image and additional images via .*$', re.IGNORECASE),
                re.compile(r'^Featured image and additional images supplied$', re.IGNORECASE)
            ]
            if article_body:
                paragraphs = article_body.find_all('p')
                text_content = []
                
                for p in paragraphs:
                    if any(pattern.match(p.text.strip()) for pattern in featured_image_patterns):
                        p.decompose()
                    p_text = p.get_text().strip()
                    if p_text:
                        text_content.append(p_text)
                
                article_data["text"] = " ".join(text_content) if text_content else "Article content not found"
            else:
                article_data["text"] = "Article content not found"
        
        else:
            print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return article_data

def scrape_multiple_can_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_can_article(url)
        articles.append(article)
    return pd.DataFrame(articles)


# List of URLs to scrape
urls = ['https://www.thecanary.co/uk/news/2025/01/19/corbyn-mcdonnell-met-police/',
       'https://www.thecanary.co/uk/news/2025/01/19/filton-18-trial/',
       'https://www.thecanary.co/uk/news/2025/01/19/corbyn-met-police-palestine-march/',
       'https://www.thecanary.co/uk/analysis/2025/01/19/fii-long-covid/',
       'https://www.thecanary.co/uk/analysis/2025/01/19/starmer-new-policy/',
       'https://www.thecanary.co/uk/analysis/2025/01/17/starmer-resignation-polling/',
       'https://www.thecanary.co/uk/analysis/2025/01/17/march-for-palestine-route/',
       'https://www.thecanary.co/uk/news/2025/01/17/hastings-general-dynamics-protest/',
       'https://www.thecanary.co/uk/news/2025/01/17/government-buried-dwp-pip-report/',
       'https://www.thecanary.co/uk/analysis/2025/01/16/nhs-rcn-report/',
       'https://www.thecanary.co/uk/analysis/2025/01/16/dwp-wca-court-verdict/',
       'https://www.thecanary.co/uk/news/2025/01/16/just-stop-oil-hung-jury/',
       'https://www.thecanary.co/uk/analysis/2025/01/16/heathrow-expansion/',
       'https://www.thecanary.co/uk/news/2025/01/16/palestine-action-filton18/',
       'https://www.thecanary.co/uk/analysis/2025/01/15/pmqs-15-january/',
       'https://www.thecanary.co/uk/news/2025/01/15/everydoctor-campaign/',
       'https://www.thecanary.co/uk/analysis/2025/01/15/send-england/',
       'https://www.thecanary.co/uk/news/2025/01/15/gh-artemis/',
       'https://www.thecanary.co/uk/news/2025/01/15/shell-protest-wildfires/',
       'https://www.thecanary.co/uk/analysis/2025/01/14/ai-fourth-industrial-revolution/',
       'https://www.thecanary.co/uk/news/2025/01/14/just-stop-oil-mark-jenkinson/',
       'https://www.thecanary.co/uk/analysis/2025/01/14/anti-fatness-media-headlines/',
       'https://www.thecanary.co/uk/news/2025/01/14/leicester-birmingham-students/',
       'https://www.thecanary.co/uk/analysis/2025/01/14/palestine-action-eagle-strategic/',
       'https://www.thecanary.co/uk/news/2025/01/14/renters-rights-bill-acorn/',
       'https://www.thecanary.co/uk/news/2025/01/14/university-admissions-poorest-students/',
       'https://www.thecanary.co/uk/analysis/2025/01/14/labour-ai-policy/',
       'https://www.thecanary.co/uk/news/2025/01/13/palestine-march-18-january-bbc/',
       'https://www.thecanary.co/uk/analysis/2025/01/13/cost-of-living-skipping-meals/',
       'https://www.thecanary.co/uk/analysis/2025/01/13/scotland-access-to-justice/',
       'https://www.thecanary.co/uk/news/2025/01/13/palestine-action-parker-hannifin/',
       'https://www.thecanary.co/uk/analysis/2025/01/13/18-jan-palestine-demo-update/',
       'https://www.thecanary.co/uk/news/2025/01/13/just-stop-oil-darwin/',
       'https://www.thecanary.co/uk/analysis/2025/01/13/msm-mental-health-dwp/',
       'https://www.thecanary.co/uk/news/2025/01/12/gaie-delap-case/',
       'https://www.thecanary.co/uk/analysis/2025/01/12/bond-markets/',
       'https://www.thecanary.co/uk/news/2025/01/10/palestine-march-bbc/',
       'https://www.thecanary.co/uk/analysis/2025/01/09/schools-funding/',
       'https://www.thecanary.co/uk/analysis/2025/01/09/brexit-skills-shortage-uk/',
       'https://www.thecanary.co/uk/news/2025/01/09/just-stop-oil-abigail-percy/',
       'https://www.thecanary.co/uk/news/2025/01/09/palestine-march-18-january/',
       'https://www.thecanary.co/uk/news/2025/01/09/labour-political-donations/',
       'https://www.thecanary.co/uk/analysis/2025/01/09/dwp-complaints-report/',
       'https://www.thecanary.co/uk/analysis/2025/01/08/pmqs-8-january/',
       'https://www.thecanary.co/uk/analysis/2025/01/09/corbyn-raf-akrotiri/',
       'https://www.thecanary.co/uk/news/2025/01/08/met-police-just-blocked-a-pro-palestine-protest-from-marching-outside-the-bbc/',
       'https://www.thecanary.co/uk/analysis/2025/01/08/sas-murder/',
       'https://www.thecanary.co/uk/analysis/2025/01/08/raffi-berg-bbc/',
       'https://www.thecanary.co/uk/news/2025/01/07/just-stop-oil-dr-hart/',
       'https://www.thecanary.co/uk/analysis/2025/01/07/mcdonalds-staff-abuse/']

# Scrape articles and create a DataFrame
can_data_df = scrape_multiple_can_articles(urls)
# Store to CSV
can_data_df.to_csv("polarised_scraped_articles_can.csv", index=False)
# Print head 
can_data_df.head()

title  \
0            Corbyn and McDonnell leave police station after being INTERVIEWED UNDER CAUTION by Met   
1                       People rally at the Old Bailey in support of the Palestine Action Filton 18   
2  Jeremy Corbyn slams Met Police's wilful 'inaccuracies' following mass-arrests at Palestine march   
3      The blame game: the rise in false allegations of maternal abuse in long Covid and disability   
4                                Starmer ends a disastrous week with his most ridiculous pledge yet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

**Breitbart**

Articles have been scrapped from the News section in reverse chronological order: https://www.breitbart.com/news/source/breitbart-news/

In [ ]:
urls= ['https://www.breitbart.com/2024-election/2025/01/19/trump-makes-triumphant-crowd-entrance-at-inaugural-eve-maga-rally/',
      'https://www.breitbart.com/politics/2025/01/19/supporters-gather-for-trumps-inauguration-eve-victory-rally/',
      'https://www.breitbart.com/law-and-order/2025/01/19/biden-grants-posthumous-pardon-to-black-nationalist-marcus-garvey/',
      'https://www.breitbart.com/politics/2025/01/19/trump-swear-in-with-personal-bible-and-lincoln-bible-during-inauguration/',
      'https://www.breitbart.com/politics/2025/01/19/poll-most-americans-one-word-summary-president-joe-bidens-legacy-nothing/',
      'https://www.breitbart.com/politics/2025/01/19/exclusive-tim-scott-releases-video-highlighting-american-renewal-ahead-trump-inauguration/',
      'https://www.breitbart.com/tech/2025/01/19/tiktok-restores-services-in-u-s-after-trump-says-he-will-issue-executive-order-delaying-ban/',
      'https://www.breitbart.com/politics/2025/01/19/jon-voight-matt-boyle-james-okeefe-tribute-andrew-breitbart-patriot-awards/',
      'https://www.breitbart.com/politics/2025/01/19/poll-most-americans-have-negative-view-outgoing-president-joe-bidens-time-office/',
      'https://www.breitbart.com/entertainment/2025/01/19/actor-cnn-show-host-michael-ian-black-wants-to-fuing-impeach-trump-before-hes-sworn-in/',
      'https://www.breitbart.com/clips/2025/01/19/kaine-claims-trumps-proposed-deportations-will-be-gut-punch-to-economy/',
      'https://www.breitbart.com/clips/2025/01/19/graham-to-cbs-worry-about-reporting-the-news-fairly-which-you-dont-do-when-it-comes-to-everything-trump/',
      'https://www.breitbart.com/entertainment/2025/01/19/michael-rapaport-celebrates-the-demise-of-dirty-biased-damn-near-soft-porn-dumphole-tiktok/']

### 3. Satire
Satirical content is intended to entertain or provoke thought through humor, exaggeration, or irony. Satire is often misunderstood as factual. 

##### Features:

- Humourous or Exaggerated Tone: Content is typically marked by wit, parody, or absurdity.
- Intentional Ridiculousness: The story is meant to be funny, not factual; outlandish claims serve comedic purposes.

##### Label If:

- The piece’s goal is clearly comedic or parodic, rather than deceptive.
- The tone, language, or disclaimers indicate it’s intentionally satirical.

##### Do Not Label If:

- The piece uses humour but is still intended to mislead (label as Fabricated Content).
- The piece is comedic but still pushing a heavily skewed narrative as if it’s true (label as Polarised Content).

##### Sources:
- The Onion (USA - 55 articles)
- Babylon Bee (USA - 50 articles)
- The Daily Squib (UK - 45 articles)
- Waterford Whispers (IE - 50 articles)


**The Onion**

The articles scraped are the ones featured on the 2024 "Annual Year" post found here: https://theonion.com/our-annual-year-2024/ - the top 5 from each month have been chosen (image posts have been excluded as per scope), so a total of 55 articles a December has not been included in their roundup.

In [2]:
def scrape_onion_article(url):
    """
    Scrapes an article from a given URL on theonion.com and extracts relevant information.

    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Satire", #satire is hardcoded here as we know TheOnion is a satire site
        "url": url
    }

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Title
        title_meta = soup.find('meta', property='og:title')
        article_data["title"] = title_meta['content'] if title_meta else "Title not found"
        
        # URL
        url_meta = soup.find('meta', property='og:url')
        article_data["url"] = url_meta['content'] if url_meta else url  # Fallback to input URL
        
        # Site name
        site_name_meta = soup.find('meta', property='og:site_name')
        article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
        
        # Published date
        published_date_meta = soup.find('meta', property='article:published_time')
        article_data["date"] = published_date_meta['content'] if published_date_meta else "Published date not found"
        
        # Category
        category_element = soup.find('div', class_='taxonomy-category')
        category_link = category_element.find('a') if category_element else None
        article_data["category"] = category_link.text.strip() if category_link else "Category not found"
        
        # Article copy
        content_div = soup.find(
            "div",
            {"class": lambda x: x and "entry-content" in x and "single-post-content" in x}
        )
        if content_div:
            paragraphs = content_div.find_all("p")
            full_text = " ".join(p.get_text(strip=True) for p in paragraphs)
            article_data["text"] = full_text
        else:
            article_data["text"] = "Article text not found"
    
    
    else:
        print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")
    
    return article_data


def scrape_multiple_onion_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_onion_article(url)
        articles.append(article)
    return pd.DataFrame(articles)


# List of URLs to scrape
urls = [
    #January
    "https://theonion.com/biden-addresses-nation-while-hanging-from-branch-on-sid-1851106795/",
    "https://theonion.com/marriage-counselor-sides-with-hotter-spouse-1851143488/",
    "https://theonion.com/wealthy-dad-surprises-child-with-tree-house-he-can-airb-1851112919/",
    "https://theonion.com/glowing-pulsating-hair-product-takes-control-of-gavin-1851160421/",
    "https://theonion.com/gen-z-announces-julie-andrews-is-problematic-but-refuse-1851180352/",
    #February
    "https://theonion.com/mrbeast-announces-he-has-resurrected-everyone-buried-at-1851217565/",
    "https://theonion.com/introverted-cowboy-struggling-to-round-up-posse-1851226175/",
    "https://theonion.com/country-stations-refuse-to-play-beyonce-s-music-after-a-1851261135/",
    "https://theonion.com/stab-him-stab-him-you-cowards-says-terrified-kamal-1851243467/",
    "https://theonion.com/emerging-filmmaker-malia-obama-changes-surname-to-scors-1851278946/",
    #March
    "https://theonion.com/u-s-airdrops-rubble-into-gaza-1851305713/",
    "https://theonion.com/ozempic-maker-triumphantly-announces-new-drug-that-make-1851320436/",
    "https://theonion.com/study-millennial-women-forgoing-dating-apps-in-favor-o-1851338275/",
    "https://theonion.com/beyonce-reveals-new-country-album-cover-featuring-tooth-1851355991/",
    "https://theonion.com/but-dog-likes-fighting-for-money-1851352386/",
    #April
    "https://theonion.com/finance-whiz-has-over-300-in-bank-account-1851375065/",
    "https://theonion.com/sotheby-s-announces-auction-of-napkin-on-which-jeffrey-1851375213/",
    "https://theonion.com/o-j-simpson-allowed-to-remain-living-after-coffin-does-1851403804/",
    "https://theonion.com/travis-kelce-impresses-coachella-crowd-by-tossing-taylo-1851410856/",
    "https://theonion.com/biden-carried-away-by-ants-1851422363/",
    #May
    "https://theonion.com/tesla-lays-off-entire-team-behind-brakes-1851449223/",
    "https://theonion.com/drake-drops-new-track-inviting-kendrick-lamar-out-to-co-1851458534/",
    "https://theonion.com/perdue-announces-initiative-to-even-the-playing-field-b-1851423157/",
    "https://theonion.com/new-florida-law-requires-all-women-to-produce-3-healthy-1851482288/",
    "https://theonion.com/everyone-in-er-bit-off-finger-while-holding-sandwich-1851488798/",
    #June
    "https://theonion.com/cult-leader-not-even-charismatic-1851512851/",
    "https://theonion.com/embarrassed-david-attenborough-realizes-he-spent-10-min-1851512951/",
    "https://theonion.com/newest-u-s-aid-mission-just-single-powerbar-labeled-f-1851540802/",
    "https://theonion.com/report-every-place-on-earth-has-wrong-amount-of-water-1851544516/",
    "https://theonion.com/nasa-warns-space-hawk-has-swooped-in-and-picked-up-eart-1851544578/",
    #July
    "https://theonion.com/clarence-thomas-torn-over-case-where-both-sides-offer-c-1851566812/",
    "https://theonion.com/democrats-panic-after-kamala-harris-ages-40-years-in-si-1851601473/",
    "https://theonion.com/congress-bans-roofs-1851592883/",
    "https://theonion.com/news-happening-faster-than-man-can-generate-uninformed-1851601466/",
    "https://theonion.com/god-forced-to-shave-head-after-contracting-plague-of-li-1851580149/",
    #August
    "https://theonion.com/environmentalists-warn-u-s-running-out-of-small-wooded-1851609190/",
    "https://theonion.com/r-kelly-petitions-supreme-court-to-watch-him-pee-1851619802rev1723482404693/",
    "https://theonion.com/federated-union-of-bear-cub-carcass-dumpers-endorses-rf-1851613425/",
    "https://theonion.com/glen-powell-opens-up-about-dangerous-stunt-work-filming-with-sydney-sweeneys-breasts/",
    "https://theonion.com/j-d-vance-accuses-tim-walz-of-stolen-valor-for-wearing-1851621120/",
    #September
    "https://theonion.com/everyone-in-restaurant-jealous-of-toddler-who-gets-to-wear-pajamas-and-watch-ipad/",
    "https://theonion.com/horrified-taylor-swift-realizes-football-happens-every-year/",
    "https://theonion.com/trump-avoids-answering-hard-questions-by-pretending-he-shot-in-ear-again/",
    "https://theonion.com/man-replies-stop-to-political-fundraiser-text-like-powerful-wizard-casting-spell-to-ward-off-mythical-beast/",
    "https://theonion.com/scarecrow-has-double-ds/",
    #October
    "https://theonion.com/the-onion-officially-endorses-joe-biden-for-president/",
    "https://theonion.com/texas-sex-ed-class-teaches-boys-how-to-cheat-on-pregnant-wife/",
    "https://theonion.com/sabrina-carpenter-completes-mandatory-service-in-south-korean-military/",
    "https://theonion.com/north-carolina-family-informed-their-insurance-policy-voided-once-house-gets-wet/",
    "https://theonion.com/grandma-who-survived-great-depression-casually-drops-that-she-once-killed-man-for-mayonnaise/",
    #November
    "https://theonion.com/piss-soaked-tucker-carlson-claims-demon-urinated-on-him-while-he-slept/",
    "https://theonion.com/trump-calls-harris-to-congratulate-himself-on-winning/",
    "https://theonion.com/america-defeats-america/",
    "https://theonion.com/man-forgetting-difference-between-meteoroid-meteorite-struggles-to-describe-what-just-killed-his-dog/",
    "https://theonion.com/every-movement-in-mans-burrito-eating-technique-informed-by-past-burrito-tragedies/"
]

# Scrape articles and create a DataFrame
onion_data_df = scrape_multiple_onion_articles(urls)
# Store to CSV
onion_data_df.to_csv("satire_scraped_articles_onion.csv", index=False)
# Print head 
onion_data_df.head()

,title,text,site,date,category,class,url
0,Biden Addresses Nation While Hanging From Branch On Side Of Cliff,"WASHINGTON—Using his platform to plead for Americans to lend him a hand, President Joe Biden addressed the nation Monday while hanging from a branch on the side of a cliff. “Our democracy has never before hung in the balance more than it has at this moment when I am in danger of plummeting 50 feet to those sharp rocks below,” said Biden, who implored the U.S. populace to set aside its differences and find a long stick, a rope, or, preferably, a helicopter that they could use to return him to stable ground. “What’s important is not what led us to this point, but rather how we choose to move forward in helping me back up. Even a carefully placed mattress or pile of sofa cushions would do. My fellow Americans, I urge you to act fast, as a small bird has landed on my head and is now pecking at me.” At press time, a Gallup Poll had found that 70% of Americans opposed Biden being rescued.",The Onion,2024-01-01T11:45:00+00:00,Politics,Satire,https://theonion.com/biden-addresses-nation-while-hanging-from-branch-on-sid-1851106795/
1,Marriage Counselor Sides With Hotter Spouse,"ANCHORAGE, AK—Stating that she had heard both perspectives and could understand their frustrations, marriage counselor Laurie Hartford reportedly told couple David and Julia Carter that she ultimately had to side with the hotter spouse. “So, I’ve listened to everything you’ve had to say, and I’ve come to the conclusion that while David does seem to be emotionally withholding, he’s also at least two points hotter,” said the therapist, who rushed to note that, in all fairness, she needed to take into consideration that she would at best describe the female half of the relationship “as, like, a six even on her best day.” “I’ve spent hours listening to you pour out your hearts and that’s never easy, so pat yourselves on the back. But, frankly, only one of you has bothered to comb your hair or put on a nice shirt at these sessions. I’m not in any way trying to invalidate your experiences. All I’m saying is that only one of you—David—has an ass that you could bounce a quarter off, and the other one is kind of an uggo, if that makes sense?” Hartford went on to say that it might be helpful if Julia stayed at home for their next sessions so that they could spend more time understanding where, exactly, David’s hotness came from.",The Onion,2024-01-09T11:30:00+00:00,Local,Satire,https://theonion.com/marriage-counselor-sides-with-hotter-spouse-1851143488/
2,Wealthy Dad Surprises Child With Tree House He Can Airbnb For Passive Income,"WILMETTE, IL—Telling the child not to peek as they walked into the backyard, local wealthy man Kenneth Schweitz reportedly surprised his son Tuesday with a tree house that the young boy could Airbnb for passive income. “It’s time you got your own little space that can be rented out for short-term stays and used to produce a reliable revenue stream,” a visibly excited Schweitz said as he took his hands off his son’s eyes to reveal the fully appointed structure built into the tree’s branches, stressing to the boy that he would not have to do any real work for the lodging to generate substantial returns. “Your mom and I can help you decorate it, but then it’s all up to you to decide how much to charge per night and which cleaning service to hire, bud. After that, you can sit back and collect thousands of dollars a month. How cool is that? You and your little friends are going to have so much fun building your little real estate empire. Enjoy!” At press time, sources reported Schweitz’s son was enthusiastically climbing into the tree house to serve an eviction notice to the low-income family currently living there.",The Onion,2024-01-09T17:30:00+00:00,Local,Satire,https://theonion.com/wealthy-dad-surprises-child-with-tree-house-he-can-airb-1851112919/
3,"Glowing, Pulsating Hair Product Takes Control Of Gavin Newsom’s Thoughts","SACRAMENTO, C

**Babylon Bee**

The top 50 articles from the Greatest Hits page (https://babylonbee.com/news?sort=greatest-hits) have been scraped. The categories "Christian Living" and "Scripture" were excluded for being too niche. 


In [3]:
urls = [
    "https://babylonbee.com/news/trump-i-have-done-more-for-christianity-than-jesus",
    "https://babylonbee.com/news/senate-to-be-replaced-with-room-full-of-monkeys-throwing-feces",
    "https://babylonbee.com/news/motorcycle-that-identifies-as-bicycle-sets-world-cycling-record",
    "https://babylonbee.com/news/trumps-says-5-golden-tickets-to-be-hidden-among-stimulus-checks",
    "https://babylonbee.com/news/nfl-to-adorn-all-uniforms-with-lace-doilies-in-to-honor-rbg",
    "https://babylonbee.com/news/pelosi-rips-up-bible",
    "https://babylonbee.com/news/biden-cuts-holes-in-medical-mask-so-he-can-still-sniff-people",
    "https://babylonbee.com/news/man-identifying-6-year-old-crushes-game-winning-homer-tee-ball-championship",
    "https://babylonbee.com/news/biden-i-am-the-only-candidate-who-can-beat-ronald-reagan",
    "https://babylonbee.com/news/fisher-price-introduces-supreme-court-protest-playhouse-that-can-be-vandalized-and-burned-down",
    "https://babylonbee.com/news/cracker-jacks-changes-name-to-more-politically-correct-caucasian-jacks",
    "https://babylonbee.com/news/cdc-people-dirt-clintons-843-greater-risk-suicide",
    "https://babylonbee.com/news/walmart-requiring-all-shoppers-to-wear-pants",
    "https://babylonbee.com/news/ilhan-omar-withdraws-support-from-bill-to-save-the-earth-after-learning-thats-where-israel-is",
    "https://babylonbee.com/news/inspiring-celebrities-spell-out-were-all-in-this-together-with-their-yachts",
    "https://babylonbee.com/news/democrats-warn-that-american-people-may-tamper-with-next-election",
    "https://babylonbee.com/news/people-who-tweet-in-support-of-foreign-wars-to-be-automatically-enlisted-in-armed-forces",
    "https://babylonbee.com/news/bernie-sanders-praises-china-for-eradicating-poverty-by-killing-all-the-poor-people",
    "https://babylonbee.com/news/pence-cancels-general-election-to-stymie-coronavirus",
    "https://babylonbee.com/news/walmart-discontinues-sale-of-auto-parts-to-prevent-car-accidents",
    "https://babylonbee.com/news/federal-prison-hires-top-rated-italian-bodyguard-hillena-clintonelli-to-protect-ghislaine-maxwell",
    "https://babylonbee.com/news/kim-jong-un-attends-ivy-league-university-to-learn-new-brainwashing-techniques",
    "https://babylonbee.com/news/florida-recount-finally-wraps-up-al-gore-declared-president",
    "https://babylonbee.com/news/powerful-protesters-spell-out-love-with-burning-homes-and-businesses",
    "https://babylonbee.com/news/joel-osteen-tests-positive-for-heresy",
    "https://babylonbee.com/news/caravan-of-liberal-americans-makes-way-toward-socialist-paradise-of-venezuela",
    "https://babylonbee.com/news/in-genius-move-trump-supports-impeachment-forcing-democrats-to-oppose-it",
    "https://babylonbee.com/news/cnn-publishes-real-news-story-for-april-fools-day",
    "https://babylonbee.com/news/government-accidentally-shuts-itself-down-with-ban-on-non-essential-businesses",
    "https://babylonbee.com/news/wife-unaware-that-movie-will-answer-all-her-questions-if-she-just-pays-attention",
    "https://babylonbee.com/news/bernie-sanders-arrives-in-hong-kong-to-lecture-protesters-on-how-good-they-have-it-under-communism",
    "https://babylonbee.com/news/jussie-smollett-offered-job-at-cnn-after-fabricating-news-story-out-of-thin-air",
    "https://babylonbee.com/news/portland-police-wish-there-were-some-kind-of-organized-armed-force-that-could-fight-back-against-antifa",
    "https://babylonbee.com/news/to-celebrate-move-to-texas-tesla-introduces-battery-powered-ar-15",
    "https://babylonbee.com/news/genius-trump-nominates-joe-biden-to-supreme-court",
    "https://babylonbee.com/news/hillary-clinton-accidentally-posts-condolences-for-tulsi-gabbards-suicide-one-day-early",
    "https://babylonbee.com/news/twitter-shuts-down-entire-network-to-slow-spread-of-negative-biden-news",
    "https://babylonbee.com/news/celebrities-show-solidarity-with-protesters-by-burning-their-own-homes-to-the-ground",
    "https://babylonbee.com/news/lego-introduces-new-sharper-bricks-that-instantly-kill-you-when-you-step-on-them",
    "https://babylonbee.com/news/democrats-call-for-flags-to-be-flown-half-mast-to-grieve-death-of-soleimani",
    "https://babylonbee.com/news/californians-brace-for-deadly-50-degree-cold-front",
    "https://babylonbee.com/news/brilliant-trump-puts-himself-on-all-postage-stamps-forcing-democrats-to-abolish-the-usps",
    "https://babylonbee.com/news/nations-nerds-wake-up-in-utopia-where-everyone-stays-inside-sports-canceled-social-interaction-forbidden",
    "https://babylonbee.com/news/hollywood-rushes-to-make-pedophilia-acceptable-before-theyre-outed-by-ghislaine-maxwell",
    "https://babylonbee.com/news/as-part-of-settlement-with-nick-sandmann-cnn-hosts-must-wear-maga-hats-while-on-the-air",
    "https://babylonbee.com/news/biden-campaign-says-he-is-so-close-to-a-vp-pick-he-can-smell-her",
    "https://babylonbee.com/news/trump-says-to-drink-lots-of-water-media-reports-as-deranged-trump-tells-everyone-to-drown-themselves",
    "https://babylonbee.com/news/starbucks-unveils-new-satanic-holiday-cups",
    "https://babylonbee.com/news/bill-clinton-allegations-of-sexual-misconduct-should-disqualify-a-man-from-public-office",
    "https://babylonbee.com/news/joel-osteen-launches-line-pastoral-wear-sheeps-clothing"
]

def scrape_bee_article(url):
    """
    Scrapes an article from a given URL on babylonbee.com and extracts relevant information.

    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Satire", #satire is hardcoded here as we know BabylonBee is a satire site
        "url": url
    }

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Title
        title_meta = soup.find('meta', property='og:title')
        article_data["title"] = title_meta['content'] if title_meta else "Title not found"
        
        # URL
        url_meta = soup.find('meta', property='og:url')
        article_data["url"] = url_meta['content'] if url_meta else url  # Fallback to input URL
        
        # Site name
        site_name_meta = soup.find('meta', property='og:site_name')
        article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
        
        # Published date       
        published_date_meta = soup.find('meta', {"name": "published_at"})
        if published_date_meta and published_date_meta.get("content"):
            article_data["date"] = published_date_meta["content"].split()[0]
        else: "Published date not found"
        
        # Category
        category_link = soup.find("a", href=lambda href: href and "/news/categories/" in href)
        if category_link:
            article_data["category"] = category_link.get_text(strip=True)
        else:
            article_data["category"] = "Category not found"
            
        # Article copy
        content_div = soup.find("div", class_="text-lg mt-6 leading-6 text-gray-700 article-content mx-2 sm:mx-0")
        if content_div:
            paragraphs = content_div.find_all("p")
            full_text = " ".join(p.get_text(strip=False) for p in paragraphs)
            article_data["text"] = full_text.strip()
        else:
            article_data["text"] = "Article text not found"
    
    else:
        print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")
    
    return article_data


def scrape_multiple_bee_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_bee_article(url)
        articles.append(article)
    return pd.DataFrame(articles)

# Scrape articles and create a DataFrame
bee_data_df = scrape_multiple_bee_articles(urls)
# Store to CSV
bee_data_df.to_csv("satire_scraped_articles_bee.csv", index=False)
# Print df 
bee_data_df.head()

,title,text,site,date,category,class,url
0,Trump: 'I Have Done More For Christianity Than Jesus',"WASHINGTON, D.C. - In response to the Christianity Today editorial calling for his removal, Trump called the magazine a ""left-wing rag"" and said, ""I have done more for Christianity than Jesus."" ""I mean, the name of the magazine is Christianity Today, and who is doing more for Christians today? Not Jesus. He disappeared; no one knows what happened to him. But I'm out there every day protecting churches from crazy liberals."" While Trump admitted that Jesus did do some things for Christianity in the past, Trump said he was doing more now and it was more substantial. ""I'm appointing judges to help protect religious rights,"" Trump stated. ""How many judges has Jesus appointed? He says something about judging people in the future, but I ain't seen it."" Furthermore, Trump asserted that he ""saved Christmas."" ""Look what I've done,"" he said. ""You can say 'Merry Christmas' now. In fact, if you say 'Happy Holidays' and don't immediately make it clear you're referring to Christmas, you go to prison. What has Jesus ever done for Christmas? Be born? He wants credit for that? Come on.""",The Babylon Bee,2019-12-23,Politics,Satire,https://babylonbee.com/news/trump-i-have-done-more-for-christianity-than-jesus
1,Senate To Be Replaced With Room Full Of Monkeys Throwing Feces,"WASHINGTON, D.C. - In an emergency, overnight referendum, the American people voted on Thursday to replace the United States Senate with a room full of monkeys throwing feces. The measure passed with 57% of the vote. 22% of voters thought the Senate should be replaced by barking seals, while 17% voted that the replacement should be the pit of venomous snakes from Indiana Jones. 3.97% voted that Senate members be replaced by screaming goats. ""About 100 people"" voted for the current Senators to keep their jobs, with this tiny voting bloc centered in Washington, D.C. Highland Ape Rescue out of West Virginia will be teaming up with Cornwell Primate farms to supply hundreds of monkeys and apes to the Senate. The animals will be fed a nutritious mixture of foods that produce easily throwable feces. Protective glass will be put up around the Senate for camera crews to safely film, but anyone being interviewed by the new senators will have to sit in the middle of the poo-flinging octagon, coming under a heavy barrage of projectile excrement. ""It will be a huge improvement from how things were before,"" said ape trainer, Marlena Henwick. ""No more 10-12 hour hearings. With these monkeys, all the fecal projectiles will have been flung in under 30 minutes. One and done."" The recently replaced senators will be placed on display at the National Zoo in Washington, D.C. for families to observe and zoologists to study.",The Babylon Bee,2018-09-28,Politics,Satire,https://babylonbee.com/news/senate-to-be-replaced-with-room-full-of-monkeys-throwing-feces
2,Motorcyclist Who Identifies As Bicyclist Sets Cycling World Record,"NEW YORK, NY - In an inspiring story from the world of professional cycling, a motorcyclist who identifies as a bicyclist has crushed all the regular bicyclists, setting an unbelievable world record. In a local qualifying race for the World Road Cycling League, the motorcyclist crushed the previous 100-mile record of 3 hours, 13 minutes with his amazing new score of well under an hour. Professional motorcycle racer Judd E. Banner, the brave trans-vehicle rider, was allowed to race after he told league organizers he's always felt like a bicyclist in a motorcyclist's body. ""Look, my ride has handlebars, two wheels, and a seat,"" he told reporters as he accepted a trophy for his incredible time trial. ""Just because I've got a little extra hardware, such as an 1170-cc flat-twin engine with 110 horsepower, doesn't mean I have any kind of inherent advantage here."" Banner also said he painted the word ""HUFFY"" on the side of his bike, ensuring he has no advantage 

**The Daily Squib**

45 articles were taken from the "Most Popular" page: https://www.dailysquib.co.uk/category/most-popular

In [4]:
urls = [
    "https://www.dailysquib.co.uk/entertainment/58255-bonus-good-newsfor-american-men-unhinged-batsht-crazy-liberal-women-going-celibate.html",
    "https://www.dailysquib.co.uk/world/56316-rachel-maddow-concerned-trump-will-put-her-in-fema-camp-during-second-term-yes-im-worried.html",
    "https://www.dailysquib.co.uk/entertainment/55825-im-a-cruise-ship-worker-these-are-the-six-things-smart-passengers-always-do-onboard.html",
    "https://www.dailysquib.co.uk/world/56106-labour-plan-to-have-speakers-and-listening-devices-on-every-lamp-post.html",
    "https://www.dailysquib.co.uk/sci_tech/55812-personal-computers-and-smartphones-were-introduced-for-benefit-of-ai.html",
    "https://www.dailysquib.co.uk/entertainment/55590-analysis-was-prince-harry-making-a-statement-in-latest-address.html",
    "https://www.dailysquib.co.uk/world/54018-interconnected-the-internet-only-creates-war-for-humanity.html",
    "https://www.dailysquib.co.uk/world/48511-world-economic-forum-brutal-totalitarian-communist-china-is-model-for-western-nations.html",
    "https://www.dailysquib.co.uk/entertainment/48277-matt-hancock-found-with-huge-amounts-of-midazolam-in-jungle.html",
    "https://www.dailysquib.co.uk/world/47729-the-beginning-of-the-post-consumerist-era.html",
    "https://www.dailysquib.co.uk/world/45624-netflix-harry-and-meghan-enjoy-themselves-exploiting-disabled-veterans-for-cash.html",
    "https://www.dailysquib.co.uk/entertainment/41634-experts-meghan-markle-thought-she-could-move-up-rank-in-royal-family.html",
    "https://www.dailysquib.co.uk/entertainment/41343-boo-hoo-you-made-me-cry.html",
    "https://www.dailysquib.co.uk/world/41322-is-harry-now-a-national-security-threat.html",
    "https://www.dailysquib.co.uk/entertainment/41208-first-transgender-woman-crowned-miss-minnesota-2021.html",
    "https://www.dailysquib.co.uk/entertainment/41200-meghan-markle-endures-bird-shit-trauma-during-oprah-interview.html",
    "https://www.dailysquib.co.uk/world/41113-meghan-markle-demands-english-county-of-sussex-is-moved-to-california.html",
    "https://www.dailysquib.co.uk/entertainment/39995-queen-meghan-and-king-harry-of-america-knight-their-gardener.html",
    "https://www.dailysquib.co.uk/world/39382-hunter-biden-i-cant-wait-to-move-into-white-house-to-smoke-crack-rocks.html",
    "https://www.dailysquib.co.uk/world/39255-obama-and-hunter-biden-sold-out-america-to-the-highest-bidder-china.html",
    "https://www.dailysquib.co.uk/world/39205-the-biden-incest-plot-thickens.html",
    "https://www.dailysquib.co.uk/entertainment/38604-keeping-up-with-the-sussexes-netflix-series-coming-in-december.html",
    "https://www.dailysquib.co.uk/world/38599-trump-to-open-presidential-library-of-authors-books-written-about-how-bad-he-is.html",
    "https://www.dailysquib.co.uk/entertainment/38512-meghan-is-imitating-diana-after-years-of-study.html",
    "https://www.dailysquib.co.uk/entertainment/38263-another-tedious-megan-markle-lecture-to-the-unwashed-masses.html",
    "https://www.dailysquib.co.uk/entertainment/38146-meghan-markle-pees-in-extensive-gardens-instead-of-using-16-bathrooms.html",
    "https://www.dailysquib.co.uk/world/38067-ironic-that-blm-antifa-heroes-karl-marx-and-engels-thought-blacks-closer-to-animal-kingdom-than-whites.html",
    "https://www.dailysquib.co.uk/world/38023-why-is-michelle-obama-so-depressed.html",
    "https://www.dailysquib.co.uk/world/37706-blm-campaign-failure-are-black-people-more-hated-now-than-before-riots.html",
    "https://www.dailysquib.co.uk/world/37612-mount-rushmore-presidents-could-be-replaced-by-blm-and-metoo-founders.html",
    "https://www.dailysquib.co.uk/world/37341-blm-doing-to-white-people-what-the-nazis-did-to-jews-dehumanizing-them.html",
    "https://www.dailysquib.co.uk/world/37294-george-floyd-to-be-sainted-by-pope-in-america.html",
    "https://www.dailysquib.co.uk/world/37236-intelligence-china-encouraging-blm-antifa-rioters-across-u-s-cities.html",
    "https://www.dailysquib.co.uk/world/36763-civilian-harry-misses-his-old-life-and-regrets-listening-to-meghan-markle.html",
    "https://www.dailysquib.co.uk/world/36586-meghan-markle-appealing-to-trump-to-end-coronavirus-pandemic-because-her-headlines-are-gone.html",
    "https://www.dailysquib.co.uk/entertainment/36508-archehole-harry-and-meghan-reveal-new-money-making-venture.html",
    "https://www.dailysquib.co.uk/world/35810-coronavirus-cui-bono-who-benefits.html",
    "https://www.dailysquib.co.uk/world/35975-meghan-reveals-harry-suffers-from-post-traumatic-royal-disorder.html",
    "https://www.dailysquib.co.uk/world/35896-will-harry-ever-forgive-meghan-for-her-crime.html",
    "https://www.dailysquib.co.uk/entertainment/35738-disney-could-replace-meghan-markle-with-plank-of-wood.html",
    "https://www.dailysquib.co.uk/entertainment/35629-defiant-meghan-markle-vs-windsor-royal-family.html",
    "https://www.dailysquib.co.uk/entertainment/35605-thomas-markle-bans-meghan-and-harry-from-using-markle-brand.html",
    "https://www.dailysquib.co.uk/world/35419-chinese-water-supply-contains-faecal-matter-aiding-spread-of-coronavirus.html",
    "https://www.dailysquib.co.uk/world/35341-chinese-authorities-misreporting-coronavirus-deaths.html",
    "https://www.dailysquib.co.uk/world/35315-remainer-tears-to-be-used-to-generate-electricity-for-britain.html",
]

def scrape_squib_article(url):
    """
    Scrapes an article from a given URL on dailysquib.co.uk and extracts relevant information.

    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Satire", #satire is hardcoded here as we know the Daily Squib is a satire site
        "url": url
    }

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # title
        title_meta = soup.find('meta', property='og:title')
        article_data["title"] = title_meta['content'] if title_meta else "Title not found"
        
        # URL
        url_meta = soup.find('meta', property='og:url')
        article_data["url"] = url_meta['content'] if url_meta else url  # Fallback to input URL
        
        # Site name
        site_name_meta = soup.find('meta', property='og:site_name')
        article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
        
        # Published date        
        published_meta = soup.find("meta", property="article:published_time")
        if published_meta and published_meta.get("content"):
            article_data["date"] = published_meta["content"].split("T")[0]
        
        # Category
        category_div = soup.find("div", class_="tdb-category td-fix-index")
        if category_div:
            cat_links = category_div.find_all("a", class_="tdb-entry-category")
            if cat_links:
                categories = [
                    #ignore "most popular"
                    a.get_text(strip=True) for a in cat_links if a.get_text(strip=True).lower() != "most popular"
                ]  
                #if multiple categories, return the first
                article_data["category"] = categories[0]
            else:
                article_data["category"] = "Category not found"
        else:
            article_data["category"] = "Category not found"

        # Extract the article text
        content_div = soup.find("div", class_="td-post-content")
        
        if content_div:
            # remove blockquotes (e.g. embedded tweets)
            for bq in content_div.find_all("blockquote"):
                bq.decompose()
            paragraphs = content_div.find_all("p")
            full_text = " ".join(p.get_text(strip=False) for p in paragraphs)
            article_data["text"] = full_text.strip()
        else:
            article_data["text"] = "Article text not found"
    
    else:
        print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")
    
    return article_data


def scrape_multiple_squib_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_squib_article(url)
        articles.append(article)
    return pd.DataFrame(articles)

# Scrape articles and create a DataFrame
squib_data_df = scrape_multiple_squib_articles(urls)
# Store to CSV
squib_data_df.to_csv("satire_scraped_articles_squib.csv", index=False)
# Print df 
squib_data_df

title  \
0                   Bonus Good News For American Men - Unhinged Batsh*t Crazy Liberal Women Going Celibate   
1         Rachel Maddow Concerned Trump Will Put Her in ‘FEMA Camp’ During Second Term: ‘Yes, I’m Worried’   
2                   I'm a Cruise Ship Worker...These are the SIX Things Smart Passengers Always do Onboard   
3                                    Labour Plan to Have Speakers and Listening Devices on Every Lamp Post   
4                                     Personal Computers and Smartphones Were Introduced For Benefit of AI   
5                                         ANALYSIS: Was Prince Harry Making a Statement in Latest Address?   
6                                               Interconnected: The Internet Only Creates War For Humanity   
7                 World Economic Forum: "Brutal Totalitarian Communist China is Model For Western Nations"   
8                                              Matt Hancock Found With Huge Amounts of Midazolam in Jungle   
9                                                                The Beginning of the Post Consumerist Era   
10                         Netflix Harry and Meghan Enjoy Themselves Exploiting Disabled Veterans For Cash   
11                                   Experts: Meghan Markle Thought She Could Move Up Rank in Royal Family   
12                                                                             "Boo Hoo! You Made Me Cry!"   
13                                                                Is Harry Now a National Security Threat?   
14                                                     First Transgender Woman Crowned Miss Minnesota 2021   
15                                           Meghan Markle Endures Bird Shit Trauma During Oprah Interview   
16                                   Meghan Markle Demands English County of Sussex is Moved to California   
17                                            Queen Meghan and King Harry of America Knight Their Gardener   
18                              Hunter Biden: "I Can't Wait to Move Into White House to Smoke Crack Rocks"   
19                                  Obama and Hunter Biden: Sold Out America to the Highest Bidder - CHINA   
20                                                                          The Biden Incest Plot Thickens   
21                                        Keeping Up With the Sussexes - Netflix Series Coming in December   
22                         Trump to Open Presidential Library of Authors Books Written About How Bad He is   
23                                                          Meghan is Imitating Diana After Years of Study   
24                                            Another Tedious Megain Markle Lecture to the Unwashed Masses   
25                                   Meghan Markle Pees in Extensive Gardens Instead of Using 16 Bathrooms   
26  Ironic That BLM ANTIFA Heroes Karl Marx and Engels Thought Blacks Closer to Animal Kingdom Than Whites   
27                                                                     Why is Michelle Obama So Depressed?   
28                                BLM Campaign Failure: Are Black People More Hated Now Than Before Riots?   
29                                  Mount Rushmore Presidents Could Be Replaced by BLM and #MeToo Founders   
30                                BLM Doing to White People What the Nazis Did to Jews - Dehumanizing Them   
31                                                           George Floyd to be Sainted by Pope in America   
32                                   Intelligence: China Encouraging BLM ANTIFA Rioters Across U.S. Cities   
33                               Civilian Harry Misses His Old Life and Regrets Listening to Meghan Markle   
34             Meghan Markle Appealing to Trump to End Coronavirus Pandemic Because Her Headlines Are Gone   
35                                             Archehole: Harry and Meghan Reveal New Money Making Venture   
36                             

**Waterford Whispers**

50 articles were take from the homepage (https://waterfordwhispersnews.com/), sorted from most recent to least recent.

In [5]:
urls = [
    "https://waterfordwhispersnews.com/2025/01/06/dickhead-boss-wants-to-hit-the-ground-running-in-2025/",
    "https://waterfordwhispersnews.com/2025/01/03/organised-local-woman-straight-onto-revenue-portal-to-get-that-e4-25-tax-back-shes-owed/",
    "https://waterfordwhispersnews.com/2025/01/06/eight-additional-data-centres-needed-to-store-pictures-of-irish-snow-energy-watchdog-warns/",
    "https://waterfordwhispersnews.com/2025/01/06/colin-farrell-asks-if-three-golden-globes-are-redeemable-for-one-oscar/",
    "https://waterfordwhispersnews.com/2025/01/06/remote-workers-wouldnt-have-agreed-to-work-from-home-if-they-knew-it-meant-zero-snow-days-off/",
    "https://waterfordwhispersnews.com/2025/01/06/im-too-old-and-rich-to-be-righteous-anymore-bono/",
    "https://waterfordwhispersnews.com/2025/01/02/irish-couples-under-increasing-pressure-to-have-minimoon/",
    "https://waterfordwhispersnews.com/2025/01/01/5-realistic-new-years-resolutions/",
    "https://waterfordwhispersnews.com/2024/12/30/seeing-elder-millennials-in-news-headlines-like-a-stab-in-the-heart/",
    "https://waterfordwhispersnews.com/2024/12/26/nations-traffic-at-standstill-as-post-christmas-re-turn-machine-queues-clog-roads/",
    "https://waterfordwhispersnews.com/2024/12/24/christmas-miracle-fine-gael-fianna-fail-put-aside-differences-to-play-football-in-leinster-house-trenches/",
    "https://waterfordwhispersnews.com/2024/12/22/local-woman-cant-believe-how-many-bullshit-made-up-christmas-traditions-in-laws-have/",
    "https://waterfordwhispersnews.com/2024/12/20/how-come-you-your-dad-support-different-teams-innocently-asks-girlfriend-about-to-receive-crash-course-in-chelseas-early-2000s-transformation/",
    "https://waterfordwhispersnews.com/2024/12/19/investigation-launched-to-discover-why-sofas-no-longer-come-with-arm-rest-covers/",
    "https://waterfordwhispersnews.com/2024/12/19/tesco-expand-self-service-checkout-to-include-customer-stacking-shelves-processing-deliveries/",
    "https://waterfordwhispersnews.com/2024/12/18/fears-rip-ie-death-notice-charge-may-turn-irish-funerals-into-a-money-racket/",
    "https://waterfordwhispersnews.com/2024/12/18/martina-burke-starting-to-suspect-family-would-do-anything-to-get-away-from-her/",
    "https://waterfordwhispersnews.com/2024/12/18/people-insisting-on-posting-about-death-destruction-in-the-birthplace-of-jesus-asked-to-shut-up-so-we-can-all-enjoy-a-guilt-free-christmas/",
    "https://waterfordwhispersnews.com/2024/12/18/fresh-hope-irish-politics-changing-for-better-with-ff-fg-supporting-td-who-thinks-3-year-old-immigrants-are-in-isis-for-ceann-comhairle/",
    "https://waterfordwhispersnews.com/2024/12/18/on-this-day-1981-irelands-first-swingers-discuss-having-no-one-to-ride/",
    "https://waterfordwhispersnews.com/2024/12/17/what-irish-people-are-saying-about-israel-closing-its-embassy-in-ireland/",
    "https://waterfordwhispersnews.com/2024/12/18/4-year-old-pours-glass-of-ribena-after-another-exhausting-day/",
    "https://waterfordwhispersnews.com/2024/12/16/mcentee-reassures-public-dublin-city-safe-between-the-times-of-10-23am-10-27am-every-second-tuesday/",
    "https://waterfordwhispersnews.com/2024/12/16/former-fine-gael-politician-charged-with-human-trafficking-possessing-sex-abuse-images-some-media-half-heartedly-reports/",
    "https://waterfordwhispersnews.com/2024/12/16/thousands-of-psychologists-descend-on-manchester-city-to-observe-study-guardiolas-meltdown/",
    "https://waterfordwhispersnews.com/2024/12/16/country-that-stole-irish-passports-for-use-in-assassinations-attacked-irish-peacekeepers-closes-embassy-in-ireland-over-countrys-opposition-to-genocide/",
    "https://waterfordwhispersnews.com/2024/12/16/conor-mcgregor-wins-brought-most-shame-to-ireland-at-rte-sports-awards/",
    "https://waterfordwhispersnews.com/2024/12/13/local-mans-always-had-a-distrust-of-the-state-ever-since-they-caught-him-doing-illegal-things/",
    "https://waterfordwhispersnews.com/2024/12/13/super-low-key-girls-xmas-meet-up-somehow-costs-e427/",
    "https://waterfordwhispersnews.com/2024/12/13/pep-guardiola-calls-shamrock-rovers-for-tips-on-winning-in-europe/",
    "https://waterfordwhispersnews.com/2024/12/12/everyone-advised-to-ignore-report-saying-house-prices-overvalued-by-10-everything-is-fine/",
    "https://waterfordwhispersnews.com/2024/12/12/man-greets-suitable-for-8-people-label-on-food-like-a-challenge/",
    "https://waterfordwhispersnews.com/2024/12/12/saudi-arabia-begins-plotting-workers-mass-grave-for-world-cup-2034/",
    "https://waterfordwhispersnews.com/2024/12/11/revenue-raise-concerns-as-some-gaa-county-board-accounts-written-in-marker-on-hurl-grip-tape/",
    "https://waterfordwhispersnews.com/2024/12/11/israel-bagsies-syria/",
    "https://waterfordwhispersnews.com/2024/12/11/luigi-mangione-remains-producers-favourite-for-next-season-of-the-bachelor/",
    "https://waterfordwhispersnews.com/2024/12/09/dont-worry-were-the-good-terrorists/",
    "https://waterfordwhispersnews.com/2024/12/11/hayes-sickened-he-didnt-hold-out-as-stocks-he-dumped-soars/",
    "https://waterfordwhispersnews.com/2024/12/09/some-irish-villages-resorting-to-cannibalism-as-power-yet-to-be-restored-after-storm-darragh/",
    "https://waterfordwhispersnews.com/2024/12/06/person-at-top-of-shop-queue-taken-completely-by-surprise-by-request-to-pay-for-goods/",
    "https://waterfordwhispersnews.com/2024/12/09/assad-to-be-given-tour-of-moscows-most-beautiful-10th-storey-windows/",
    "https://waterfordwhispersnews.com/2024/12/06/local-dad-enters-2nd-hour-of-christmas-tree-price-negotiation-stand-off/",
    "https://waterfordwhispersnews.com/2024/12/05/unitedhealthcare-board-not-sure-when-right-time-to-break-it-to-ceos-family-none-of-this-is-covered-under-insurance-plan/",
    "https://waterfordwhispersnews.com/2024/12/04/parents-counteract-child-dropping-hints-about-xmas-presents-with-dropping-hints-theyre-broke-as-fuck/",
    "https://waterfordwhispersnews.com/2024/12/04/you-up-hun-ff-fg-sends-late-night-text-to-michael-lowry/",
    "https://waterfordwhispersnews.com/2024/12/04/thats-bad-form-now-kim-jong-un-urges-south-korean-president-to-stand-down/",
    "https://waterfordwhispersnews.com/2024/12/03/supplying-weapons-to-kill-50000-people-all-fine-but-just-dont-pardon-your-son-biden-told/",
    "https://waterfordwhispersnews.com/2024/12/03/first-100-violations-of-ceasefire-are-free-us-tells-israel-as-idf-continues-to-strike-lebanon/",
    "https://waterfordwhispersnews.com/2024/12/03/fianna-fail-fine-gaels-red-lines-for-going-into-government-with-each-other/",
    "https://waterfordwhispersnews.com/2024/12/03/calm-down-sugar-tits-gregg-wallaces-guide-to-crafting-the-perfect-apology/",
    
]

def scrape_whispers_article(url):
    """
    Scrapes an article from a given URL on waterfordwhispersnews.com and extracts relevant information.

    Parameters:
    ----------
    url : str
        The URL of the article to scrape.

    Returns:
    -------
    dict
        A dictionary containing the extracted article data.
    """
    article_data = {
        "title": "",
        "text": "",
        "site": "",
        "date": "",
        "category": "",
        "class": "Satire", #satire is hardcoded here as we know WaterfordWhispers is a satire site
        "url": url
    }

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Title
        title_meta = soup.find('meta', property='og:title')
        article_data["title"] = title_meta['content'] if title_meta else "Title not found"
        
        # URL
        url_meta = soup.find('meta', property='og:url')
        article_data["url"] = url_meta['content'] if url_meta else url  # Fallback to input URL
        
        # Site name
        site_name_meta = soup.find('meta', property='og:site_name')
        article_data["site"] = site_name_meta['content'] if site_name_meta else "Site name not found"
        
        # Published date 
        date_div = soup.find("div", class_="post-date", itemprop="datePublished")
        if date_div:
            article_data["date"] = date_div.get_text(strip=True)
        else:
            article_data["date"] = "Date not found"
 
        # Category (excluding the ones used just for web display)
        excluded_categories = {"breaking news", "featured-one", "featured-two", "featured-three","homepage"}
        category_div = soup.find("div", class_="post-category")
        if category_div:
            all_cats = [a.get_text(strip=True) for a in category_div.find_all("a")]
            valid_cats = [cat for cat in all_cats
                          if cat.lower() not in excluded_categories]
            if valid_cats:
                article_data["category"] = valid_cats[0]
            else:
                article_data["category"] = "Category not found"
        else:
            article_data["category"] = "Category not found"

        # Article copy
        content_div = soup.find("div", class_="article-content", itemprop="articleBody")
        if content_div:
            for p_tag in content_div.find_all("p"):
                p_text = p_tag.get_text(strip=True).lower()
                # remove marketing snippets
                if "check out our shop." in p_text or "www.waterfordwhispers.shop" in p_text or "buy some of our merch here" in p_text or "help us to keep pissing off all the right people" in p_text:
                    p_tag.decompose()

            # remove blockquotes
            for bq in content_div.find_all("blockquote"):
                bq.decompose()

            paragraphs = content_div.find_all("p")
            full_text = " ".join(p.get_text(strip=False) for p in paragraphs)
            article_data["text"] = full_text.strip()
        else:
            article_data["text"] = "Article text not found"
    
    else:
        print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")
    
    return article_data


def scrape_multiple_whispers_articles(urls):
    """
    Scrapes multiple articles from a list of URLs and stores the data in a DataFrame.

    Parameters:
    ----------
    urls : list
        A list of article URLs to scrape.

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the scraped data from all URLs.
    """
    articles = []
    for url in urls:
        article = scrape_whispers_article(url)
        articles.append(article)
    return pd.DataFrame(articles)

# Scrape articles and create a DataFrame
whispers_data_df = scrape_multiple_whispers_articles(urls)
# Store to CSV
whispers_data_df.to_csv("satire_scraped_articles_whispers.csv", index=False)
# Print df 
whispers_data_df.head()

,title,text,site,date,category,class,url
0,Dickhead Boss Wants To Hit The Ground Running In 2025,"A COMPLETE visual replica of a gaping arsehole, local boss Jamie McCartlin, has informed staff on their first day back in 2025 that he wants to hit the ground running this year. In a circular email sent to staff that immediately led to rolling of eyes seen by WWN, McCartlin spoke of wanting ‘100%’ from staff, pointing to a less than stellar last quarter and how ‘some people around here won’t get away with coasting for another year’. “Not only is it the bastarding 2nd of January, it’s a Thursday. Save it for the first Monday of the year at the very least,” groused employee Cormac D’Arcy, who had hit the ground in 2025 and stayed there, hoping to sneak in a quick nap while no one way looking. “My internal work clock remains switched off until the Christmas Tree is fucked in the field down the road. So even if I wanted to hit, slam, bate the ground in 2025 it’ll have to wait,” offered Carmel Tullan, who didn’t make it past the first sentence in her boss’ email which began ‘New Year, New Focused Workforce’. For his part, McCartlin feels he is simply leading by example. “This ship is crew only, no passengers if you get what I’m saying. The only coasting around her is being done by the coastline down the way,” McCartlin said, communicating in empty jargon which appeared to make sense to him. Following up the email with a quick speech to the troops, McCartlin felt his message was getting through. “I won’t sugarcoat it, I need soldiers, people who eat, sleep and breathe this job. ,” concluded McCartlin, who neglected to mentioned he was offing on a two week skiing trip to Austria at the end of the month.",Waterford Whispers News,"January 6, 2025",BUSINESS,Satire,https://waterfordwhispersnews.com/2025/01/06/dickhead-boss-wants-to-hit-the-ground-running-in-2025/
1,Organised Local Woman Straight Onto Revenue Portal To Get That €4.25 Tax Back She’s Owed,"HAVING CALCULATED the projected tax relief she is entitled to under the Remote Working Relief on electricity and broadband bills at the start of December, local woman Orna Stewart is primed and ready to file her 2024 tax return. With her company allowing a flexible working arrangement for employees which includes 3 days at home, Stewart clocked up three days working from home because she thinks she gets more done in the office and she is ready to claim back what is hers. “People have no idea how much they’re entitled to, just leaving money there on the table but not me, no way Jose, that €4.25 is mine!” Stewart said as she hovered over the submit button in her sitting room, on her day off, and the weather lovely outside and all. “But since the bills are in your name, you could claim for much more, even if we split them evenly,” Stewart’s husband Martin stated as he queried why the 33-year-old wasn’t entering the full amount paid through her account, unwittingly setting himself up to be subjected to an hour’s long lecture on honesty and ethics. “Here she is, right on queue,” said the automated system in receipt of Stewarts tax declaration, making her the first person in the country to submit her declaration for the 5th year running.",Waterford Whispers News,"January 3, 2025",LOCAL NEWS,Satire,https://waterfordwhispersnews.com/2025/01/03/organised-local-woman-straight-onto-revenue-portal-to-get-that-e4-25-tax-back-shes-owed/
2,“Eight Additional Data Centres Needed To Store Pictures Of Irish Snow” Energy Watchdog Warns,"THE COMMISSION FOR Regulation of Utilities in Ireland has called on the public to refrain from taking yet more photos of the snow as server capacities reach breaking point. “Data centres already consume a huge amount of our electricity, but Meta will need to break ground on another eight data centres if you keep uploading those photos of your snowman with a carrot for a dick,” cautioned the energy watchdog. Usually limiting itself to providing a free complaint resolution service,

In [6]:
# Combine DataFrames
satire_dataset = pd.concat(
    [whispers_data_df, squib_data_df, bee_data_df, onion_data_df],
    ignore_index=True
)

# Basic checks
print(satire_dataset.info())   # Data types & non-null counts
print(satire_dataset.head())   # Quick glance at first rows

# Print out the categories
print(satire_dataset["category"].value_counts())

# Confirm 4 sites are represented
print("Number of unique sites:", satire_dataset["site"].nunique())

Number of rows in satire_dataset: 200
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     200 non-null    object
 1   text      200 non-null    object
 2   site      200 non-null    object
 3   date      200 non-null    object
 4   category  200 non-null    object
 5   class     200 non-null    object
 6   url       200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB
None
                                                                                            title  \
0                                           Dickhead Boss Wants To Hit The Ground Running In 2025   
1        Organised Local Woman Straight Onto Revenue Portal To Get That €4.25 Tax Back She’s Owed   
2    “Eight Additional Data Centres Needed To Store Pictures Of Irish Snow” Energy Watchdog Warns   
3                          Colin Farrell Asks If Three Golden Glob

In [10]:
def clean_category(cat: str) -> str:
    """
    Convert categories to lowercase, unify synonyms, and return a single standardised category.
    """
    # Convert to lowercase
    c = cat.strip().lower()
    
    # Standardise categories
    replacements = {
        'politics': 'politics',
        'local news': 'local',
        'world news': 'world',
        'world': 'world',
        'worldviews':'world',
        'entertainment': 'entertainment',
        'business': 'business',
        'health': 'health',
        'lifestyle': 'lifestyle',
        'life':'lifestyle',
        'sports': 'sports',
        'sport': 'sports',
        'football': 'sports',
        'gaa': 'sports',
        'sci/tech': 'technology',
        'celebs':'entertainment',
        'tech': 'tech',
        'u.s.':'united states',
        'uplifting viral content': 'entertainment', 
    }

    # Do the changes
    if c in replacements:
        c = replacements[c]
    return c

# Apply the cleaning
satire_dataset['category'] = satire_dataset['category'].apply(clean_category)

# Now check the new distribution
print(satire_dataset['category'].value_counts())

politics              48
entertainment         38
world                 37
local                 27
news                  19
sports                 7
lifestyle              6
business               4
health                 4
category not found     3
tech                   3
united states          3
editorials             1
Name: category, dtype: int64


In [13]:
# Store to CSV
satire_dataset.to_csv("satire_articles.csv", index=False)